In [1]:
%use coroutines

### Can run background stuff with GlobalScope.launch

In [14]:
val list = mutableListOf("Start")
var job : Job? = null
GlobalScope.launch { 
    job = launch {
        while (true) {
            delay(2500)
            list.add("foobar")
        }
    }
}

StandaloneCoroutine{Completing}@1c051051

In [16]:
list

[Start, foobar, foobar, foobar, foobar]

In [17]:
job.cancel("manual stop")

### Can repeatedly update display with loop

In [10]:
val displayId = "blockingDisplay"
DISPLAY("initial", displayId)

update #4

In [36]:
var x = 0
runBlocking { 
    while (x < 5) {
        UPDATE_DISPLAY("update #${x++}", displayId)
        delay(2500)
    }
}

### Updating display in coroutine fails

In [96]:
val globalDisplayId = "GlobalScopDisplay"
DISPLAY("init", globalDisplayId)

before delay

In [97]:
var runCount = 0
var throwable: Throwable? = null
GlobalScope.launch { 
    job = launch { 
        try {
            while (true) {
                UPDATE_DISPLAY("before delay", globalDisplayId)
                delay(2500)
                UPDATE_DISPLAY("run count: ${runCount++}", globalDisplayId)
            }
        } catch (t: Throwable) {
            throwable = t
        }
    }
}

StandaloneCoroutine{Completing}@d9a03f4

In [98]:
job

StandaloneCoroutine{Completed}@3c461982

In [99]:
runCount

1

In [100]:
throwable?.stackTrace

[jupyter.kotlin.CellExtensionsKt.getHost(CellExtensions.kt:17), jupyter.kotlin.CellExtensionsKt.UPDATE_DISPLAY(CellExtensions.kt:30), Line_102_jupyter$res102$1$1.invokeSuspend(Line_102.jupyter.kts:9), kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImpl.kt:33), kotlinx.coroutines.DispatchedTask.run(DispatchedTask.kt:104), kotlinx.coroutines.scheduling.CoroutineScheduler.runSafely(CoroutineScheduler.kt:584), kotlinx.coroutines.scheduling.CoroutineScheduler$Worker.executeTask(CoroutineScheduler.kt:811), kotlinx.coroutines.scheduling.CoroutineScheduler$Worker.runWorker(CoroutineScheduler.kt:715), kotlinx.coroutines.scheduling.CoroutineScheduler$Worker.run(CoroutineScheduler.kt:702)]

### Can save host reference but then display isn't updated

In [110]:
val fixedHostDisplayId = "fixedHost"
DISPLAY("init", fixedHostDisplayId)

before delay

In [111]:
val host = notebook.executionHost!!
var runCount = 0
var throwable: Throwable? = null
GlobalScope.launch {
    job = launch {
        try {
            while (true) {
                host.updateDisplay("before delay", fixedHostDisplayId)
                delay(2500)
                host.updateDisplay("run count: ${runCount++}", fixedHostDisplayId)
            }
        } catch (t: Throwable) {
            throwable = t
        }
    }
}

StandaloneCoroutine{Completing}@3c5b7ac1

In [112]:
job

StandaloneCoroutine{Active}@535c88df

In [115]:
runCount

5

In [116]:
job.cancel()